# Vectores Multi-modal

Estos vectores provienen del modelo de embeddings multimodal Azure AI Vision, que puede representar tanto texto como imágenes en un espacio de 1024 dimensiones.


In [ ]:
import json

with open('embeddings/images_ai-vision.json') as f:
    image_vectors = json.load(f)

with open('embeddings/sustantivos_ai-vision.json') as f:
    word_vectors = json.load(f)

In [ ]:
import random

import matplotlib.pyplot as plt

random_image_name = random.choice(list(image_vectors.keys()))
vector = image_vectors[random_image_name]

plt.bar(range(len(vector)), vector)
plt.xlabel('Dimension')
plt.ylabel('Value')
plt.show()

## Más similares a una imagen objetivo

In [ ]:
import pandas as pd

def cosine_similarity(v1, v2):
    """Calcula la similitud del coseno entre dos vectores"""
    dot_product = sum([a * b for a, b in zip(v1, v2)])
    magnitude = (sum([a**2 for a in v1]) * sum([a**2 for a in v2])) ** 0.5
    return dot_product / magnitude

def most_similar(target_vector: str, vectors: dict) -> list[list]:
    """Devuelve las imágenes más similares y sus similitudes respecto a las imágenes dadas"""
    similarities = {w: cosine_similarity(target_vector, vector) for w, vector in vectors.items()}
    most_similar = sorted(similarities, key=similarities.get, reverse=True)
    return pd.DataFrame([(vector_key, similarities[vector_key]) for vector_key in most_similar], columns=['vector key', 'similarity'])


In [ ]:
# Renderiza la imagen objetivo
target_image = "inhaleexhale_top.jpg"
plt.imshow(plt.imread(f"../product_images/{target_image}"))

In [ ]:
most_similar_df = most_similar(image_vectors[target_image], image_vectors)[0:10]
most_similar_df

In [ ]:
# Ahora renderiza cada una de esas imágenes
for image_name in most_similar_df['vector key'][1:]:
    plt.imshow(plt.imread(f'../product_images/{image_name}'))
    plt.axis('off')
    plt.show()

## Más similares a un texto

In [ ]:
target_word = "cama"
most_similar_df = most_similar(word_vectors[target_word], image_vectors)

for image_name in most_similar_df['vector key'][0:10]:
    plt.imshow(plt.imread(f'../product_images/{image_name}'))
    plt.axis('off')
    plt.show()

## Búsqueda con vectores de consulta

In [ ]:
import os

import dotenv
import requests
from azure.identity import DefaultAzureCredential, get_bearer_token_provider

dotenv.load_dotenv()

azure_credential = DefaultAzureCredential()
token_provider = get_bearer_token_provider(azure_credential, "https://cognitiveservices.azure.com/.default")
AZURE_AI_VISION_ENDPOINT = os.environ["AZURE_AI_VISION_ENDPOINT"]
AZURE_AI_VISION_URL = f"{AZURE_AI_VISION_ENDPOINT}/computervision/retrieval"

def get_model_params():
    return {"api-version": "2024-02-01", "model-version": "2023-04-15"}

def get_auth_headers():
    return {"Authorization": "Bearer " + token_provider()}

def get_text_embedding(text):
    url = f"{AZURE_AI_VISION_URL}:vectorizeText"
    response = requests.post(url, headers=get_auth_headers(), params=get_model_params(), json={"text": text})
    if response.status_code != 200:
        print(response.json().get("error"))
        return None
    else:
        return response.json()["vector"]

In [ ]:
embedding = get_text_embedding("niño")
most_similar_df = most_similar(embedding, image_vectors)

for image_name in most_similar_df['vector key'][0:5]:
    plt.imshow(plt.imread(f'../product_images/{image_name}'))
    plt.axis('off')
    plt.show()

## Recursos

* [Portlandia - She's making jewelry now](https://www.youtube.com/watch?v=Kg6H0Cd1_Jc)